In [1]:
# 导入包
import pandas as pd
import gc
import numpy as np

In [2]:
# 小样本
# user_log = pd.read_csv('./data_format1/user_log_format1.csv', dtype={'time_stamp':'str'}, nrows = 1000)
# 加载全量样本
user_log = pd.read_csv('./data_format1/user_log_format1.csv', dtype={'time_stamp':'str'})
# 格式化
user_log['user_id'] = user_log['user_id'].astype(str)
# 使用merchant_id（原列名seller_id）
user_log.rename(columns={'seller_id':'merchant_id'}, inplace=True)
user_log['merchant_id'] = user_log['merchant_id'].astype(str)
user_log['item_id'] = user_log['item_id'].astype(str)
user_log['cat_id'] = user_log['cat_id'].astype(str)
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype(str)
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%m%d')
user_log['month'] = user_log['time_stamp'].astype(str).str[5:7]
user_log


,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type,month
0,328862,323294,833,2882,2661.0,1900-08-29,0,08
1,328862,844400,1271,2882,2661.0,1900-08-29,0,08
2,328862,575153,1271,2882,2661.0,1900-08-29,0,08
3,328862,996875,1271,2882,2661.0,1900-08-29,0,08
4,328862,1086186,1271,1253,1049.0,1900-08-29,0,08
...,...,...,...,...,...,...,...,...
54925325,208016,107662,898,1346,7995.0,1900-11-10,0,11
54925326,208016,1058313,898,1346,7995.0,1900-11-10,0,11
54925327,208016,449814,898,983,7995.0,1900-11-10,0,11
54925328,208016,634856,898,1346,7995.0,1900-11-10,0,11


# 数据预处理

In [3]:
user_info = pd.read_csv('./data_format1/user_info_format1.csv')
train_data1 = pd.read_csv('./data_format1/train_format1.csv')
submission = pd.read_csv('./data_format1/test_format1.csv')
train_data = pd.read_csv('./data_format2/train_format2.csv')
train_data1['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([train_data1, submission], ignore_index=True, sort=False)
matrix['label'].fillna(-1, inplace=True)
matrix.drop(['prob'], axis=1, inplace=True)
# 连接user_info表，通过user_id关联
matrix = matrix.merge(user_info, on='user_id', how='left')
matrix

,user_id,merchant_id,label,origin,age_range,gender
0,34176,3906,0.0,train,6.0,0.0
1,34176,121,0.0,train,6.0,0.0
2,34176,4356,1.0,train,6.0,0.0
3,34176,2217,0.0,train,6.0,0.0
4,230784,4818,0.0,train,0.0,0.0
...,...,...,...,...,...,...
522336,228479,3111,-1.0,test,6.0,0.0
522337,97919,2341,-1.0,test,8.0,1.0
522338,97919,3971,-1.0,test,8.0,1.0
522339,32639,3536,-1.0,test,0.0,0.0


In [4]:
matrix['age_range'].fillna(0, inplace=True)
# 0:female, 1:male, 2:unknown
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('int8')
matrix['user_id'] = matrix['user_id'].astype(str)
matrix['merchant_id'] = matrix['merchant_id'].astype(str)
del user_info, train_data1
gc.collect()
matrix

,user_id,merchant_id,label,origin,age_range,gender
0,34176,3906,0,train,6,0
1,34176,121,0,train,6,0
2,34176,4356,1,train,6,0
3,34176,2217,0,train,6,0
4,230784,4818,0,train,0,0
...,...,...,...,...,...,...
522336,228479,3111,-1,test,6,0
522337,97919,2341,-1,test,8,1
522338,97919,3971,-1,test,8,1
522339,32639,3536,-1,test,0,0


# 特征处理

In [5]:
train_data['merchant_id'] = train_data['merchant_id'].astype(str)

In [6]:
# User特征处理
user_groups = user_log.groupby(['user_id'])
# 用户交互行为数量 u1
temp = user_groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
# 使用agg 基于列的聚合操作，统计唯一值的个数 item_id, cat_id, merchant_id, brand_id
#temp = groups['item_id', 'cat_id', 'merchant_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'u2', 'cat_id':'u3', 'merchant_id':'u4', 'brand_id':'u5'})
temp = user_groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = user_groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = user_groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = user_groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')

# 时间间隔特征 u6 按照小时
temp = user_groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.days
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
# 统计操作类型为0，1，2，3的个数
temp = user_groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7',1:'u8', 2:'u9', 3:'u10'})
del temp['u7'],temp['u8']
matrix = matrix.merge(temp, on='user_id', how='left')
matrix['u7'] = matrix['u1'] - matrix['u9'] - matrix['u10']
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,u5,u6,u9,u10,u7
0,34176,3906,0,train,6,0,451,256,45,109,108,174,34.0,7.0,410.0
1,34176,121,0,train,6,0,451,256,45,109,108,174,34.0,7.0,410.0
2,34176,4356,1,train,6,0,451,256,45,109,108,174,34.0,7.0,410.0
3,34176,2217,0,train,6,0,451,256,45,109,108,174,34.0,7.0,410.0
4,230784,4818,0,train,0,0,54,31,17,20,19,163,7.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,6,0,2004,1173,71,278,282,184,26.0,208.0,1770.0
522337,97919,2341,-1,test,8,1,55,29,14,17,17,138,8.0,1.0,46.0
522338,97919,3971,-1,test,8,1,55,29,14,17,17,138,8.0,1.0,46.0
522339,32639,3536,-1,test,0,0,72,46,24,33,35,172,8.0,1.0,63.0


In [7]:
# 商家特征处理
merchant_groups = user_log.groupby(['merchant_id'])
# 商家被交互行为数量 m1
temp = merchant_groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的user_id, item_id, cat_id, brand_id 唯一值
temp = merchant_groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id':'m2', 'item_id':'m3', 'cat_id':'m4', 'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的action_type 唯一值
temp = merchant_groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
# temp['m6'] = temp['m6']+temp['m7']
del temp['m7'],temp['m6']
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 按照merchant_id 统计随机负采样的个数
temp = train_data[train_data['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
matrix['m6'] = matrix['m1'] - matrix['m8'] - matrix['m9']
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,...,u7,m1,m2,m3,m4,m5,m8,m9,m10,m6
0,34176,3906,0,train,6,0,451,256,45,109,...,410.0,16269,5819,308,20,2,410.0,961.0,2861,14898.0
1,34176,121,0,train,6,0,451,256,45,109,...,410.0,79865,10931,1179,26,2,4780.0,2699.0,4530,72386.0
2,34176,4356,1,train,6,0,451,256,45,109,...,410.0,7269,2281,67,15,2,963.0,196.0,1088,6110.0
3,34176,2217,0,train,6,0,451,256,45,109,...,410.0,60202,16870,377,5,2,3721.0,4150.0,7268,52331.0
4,230784,4818,0,train,0,0,54,31,17,20,...,NaN,48089,7500,461,27,2,2733.0,1959.0,3102,43397.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,6,0,2004,1173,71,278,...,1770.0,10105,4154,542,50,18,687.0,412.0,1982,9006.0
522337,97919,2341,-1,test,8,1,55,29,14,17,...,46.0,5543,1592,352,93,19,815.0,174.0,703,4554.0
522338,97919,3971,-1,test,8,1,55,29,14,17,...,46.0,28892,7587,272,7,2,2608.0,1588.0,3050,24696.0
522339,32639,3536,-1,test,0,0,72,46,24,33,...,63.0,14027,4956,322,19,3,793.0,398.0,2177,12836.0


In [8]:
# 按照user_id, merchant_id分组
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'}) #统计行为个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'um2', 'cat_id':'um3', 'brand_id':'um4'}) 
#统计item_id, cat_id, brand_id唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'um5', 1:'um6', 2:'um7', 3:'um8'})#统计不同action_type唯一个数
# temp['um5'] = temp['um5']+temp['um6']
del temp['um6'],temp['um5']
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.days
temp.drop(['first', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left') #统计时间间隔
matrix['um5'] = matrix['um1'] - matrix['um7'] - matrix['um8']
del temp
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,...,m10,m6,um1,um2,um3,um4,um7,um8,um9,um5
0,34176,3906,0,train,6,0,451,256,45,109,...,2861,14898.0,39,20,6,1,1.0,2.0,22,36.0
1,34176,121,0,train,6,0,451,256,45,109,...,4530,72386.0,14,1,1,1,1.0,NaN,3,NaN
2,34176,4356,1,train,6,0,451,256,45,109,...,1088,6110.0,18,2,1,1,6.0,NaN,1,NaN
3,34176,2217,0,train,6,0,451,256,45,109,...,7268,52331.0,2,1,1,1,1.0,NaN,0,NaN
4,230784,4818,0,train,0,0,54,31,17,20,...,3102,43397.0,8,1,1,1,1.0,NaN,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,6,0,2004,1173,71,278,...,1982,9006.0,5,2,1,1,1.0,NaN,1,NaN
522337,97919,2341,-1,test,8,1,55,29,14,17,...,703,4554.0,2,1,1,1,1.0,NaN,0,NaN
522338,97919,3971,-1,test,8,1,55,29,14,17,...,3050,24696.0,16,5,2,1,4.0,NaN,9,NaN
522339,32639,3536,-1,test,0,0,72,46,24,33,...,2177,12836.0,3,2,1,1,1.0,NaN,0,NaN


In [9]:
#用户购买点击比
matrix['r1'] = matrix['u9']/matrix['u7'] 
#商家购买点击比 不需要这个特征值，得分0.68
matrix['r2'] = matrix['m8']/matrix['m6'] 
#不同用户不同商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5']
matrix.fillna(0, inplace=True)
# # 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,age_2,age_3,age_4,age_5,age_6,age_7,age_8,g_0,g_1,g_2
0,34176,3906,0,train,451,256,45,109,108,174,...,0,0,0,0,1,0,0,1,0,0
1,34176,121,0,train,451,256,45,109,108,174,...,0,0,0,0,1,0,0,1,0,0
2,34176,4356,1,train,451,256,45,109,108,174,...,0,0,0,0,1,0,0,1,0,0
3,34176,2217,0,train,451,256,45,109,108,174,...,0,0,0,0,1,0,0,1,0,0
4,230784,4818,0,train,54,31,17,20,19,163,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,0,0,0,0,1,0,0,1,0,0
522337,97919,2341,-1,test,55,29,14,17,17,138,...,0,0,0,0,0,0,1,0,1,0
522338,97919,3971,-1,test,55,29,14,17,17,138,...,0,0,0,0,0,0,1,0,1,0
522339,32639,3536,-1,test,72,46,24,33,35,172,...,0,0,0,0,0,0,0,1,0,0


In [10]:
actions_at_nov = user_log[user_log['month'] == "11"]
buy_actions_at_nov = actions_at_nov[actions_at_nov['action_type'] == 2]

actions_at_oct = user_log.loc[user_log['month'] == "10"]
buy_actions_at_oct = actions_at_oct[actions_at_oct['action_type'] == 2]

actions_at_sep = user_log[user_log['month'] == "09"]
buy_actions_at_sep = actions_at_sep[actions_at_sep['action_type'] == 2]

actions_at_aug = user_log.loc[user_log['month'] == "08"]
buy_actions_at_aug = actions_at_aug[actions_at_aug['action_type'] == 2]

actions_at_jul = user_log[user_log['month'] == "07"]
buy_actions_at_jul = actions_at_jul[actions_at_jul['action_type'] == 2]

actions_at_jun = user_log.loc[user_log['month'] == "06"]
buy_actions_at_jun = actions_at_jun[actions_at_jun['action_type'] == 2]

actions_at_may = user_log.loc[user_log['month'] == "05"]
buy_actions_at_may = actions_at_may[actions_at_may['action_type'] == 2]


user_actions_at_nov = buy_actions_at_nov.groupby(['user_id', 'month'], as_index=False)['month'].agg({'user_buy_cnt_11': 'count'}).fillna(0)
del user_actions_at_nov['month']
user_actions_at_oct = buy_actions_at_oct.groupby(['user_id', 'month'], as_index=False)['month'].agg({'user_buy_cnt_10': 'count'}).fillna(0)
del user_actions_at_oct['month']
user_actions_at_sep = buy_actions_at_sep.groupby(['user_id', 'month'], as_index=False)['month'].agg({'user_buy_cnt_09': 'count'}).fillna(0)
del user_actions_at_sep['month']
user_actions_at_aug = buy_actions_at_aug.groupby(['user_id', 'month'], as_index=False)['month'].agg({'user_buy_cnt_08': 'count'}).fillna(0)
del user_actions_at_aug['month']
user_actions_at_jul = buy_actions_at_jul.groupby(['user_id', 'month'], as_index=False)['month'].agg({'user_buy_cnt_07': 'count'}).fillna(0)
del user_actions_at_jul['month']
user_actions_at_jun = buy_actions_at_jun.groupby(['user_id', 'month'], as_index=False)['month'].agg({'user_buy_cnt_06': 'count'}).fillna(0)
del user_actions_at_jun['month']
user_actions_at_may = buy_actions_at_may.groupby(['user_id', 'month'], as_index=False)['month'].agg({'user_buy_cnt_05': 'count'}).fillna(0)
del user_actions_at_may['month']

buy_actions_at_nov.head(5)

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type,month
174,328862,406349,1280,2700,5476.0,1900-11-11,2,11
429,234512,240182,81,3018,4144.0,1900-11-11,2,11
430,234512,137298,1432,3271,6957.0,1900-11-11,2,11
432,234512,179830,1208,546,2276.0,1900-11-11,2,11
460,234512,585039,825,800,6326.0,1900-11-10,2,11


In [11]:
res = pd.merge(user_actions_at_nov,user_actions_at_oct, on=['user_id'],how='left')
res = res.merge(user_actions_at_sep, on=['user_id'], how='left')
res = res.merge(user_actions_at_aug, on=['user_id'], how='left')
res = res.merge(user_actions_at_jul, on=['user_id'], how='left')
res = res.merge(user_actions_at_jun, on=['user_id'], how='left')
res = res.merge(user_actions_at_may, on=['user_id'], how='left')
# res['user_buy_deviation_between_months'] = abs(res['cnt'] - res['mean_of_num_of_actions_not_in_nov'])
res = res.fillna(0)
res

,user_id,user_buy_cnt_11,user_buy_cnt_10,user_buy_cnt_09,user_buy_cnt_08,user_buy_cnt_07,user_buy_cnt_06,user_buy_cnt_05
0,1,4,2.0,0.0,0.0,0.0,0.0,0.0
1,10,2,5.0,0.0,0.0,0.0,0.0,0.0
2,100,5,3.0,2.0,0.0,0.0,0.0,0.0
3,1000,7,0.0,0.0,0.0,0.0,0.0,0.0
4,10000,6,2.0,0.0,2.0,2.0,3.0,2.0
...,...,...,...,...,...,...,...,...
424165,99995,5,0.0,0.0,0.0,0.0,0.0,0.0
424166,99996,1,0.0,1.0,0.0,0.0,0.0,1.0
424167,99997,1,0.0,0.0,0.0,0.0,0.0,0.0
424168,99998,6,1.0,2.0,0.0,0.0,0.0,0.0


In [12]:
temp=[] 
temp=pd.DataFrame(temp)
temp['user_id'] = res['user_id']
temp['user_buy_mean_between_months'] = res.mean(axis=1)
temp['user_buy_std_between_months'] = res.std(axis=1)
temp['user_buy_min_between_months'] = res.min(axis=1)
temp['user_buy_max_between_months'] = res.max(axis=1)
temp['user_buy_sum_between_months'] = res.sum(axis=1)
temp.head(5)

,user_id,user_buy_mean_between_months,user_buy_std_between_months,user_buy_min_between_months,user_buy_max_between_months,user_buy_sum_between_months
0,1,0.857143,1.573592,0.0,4.0,6.0
1,10,1.000000,1.914854,0.0,5.0,7.0
2,100,1.428571,1.988060,0.0,5.0,10.0
3,1000,1.000000,2.645751,0.0,7.0,7.0
4,10000,2.428571,1.812654,0.0,6.0,17.0


In [13]:
# import statsmodels.formula.api as smf
# xm = np.array([1,2,3,4,5,6,7])
# xx = pd.DataFrame({"k1": xm})
# res['slope'] = None
# # res['slope'].fillna(0,inplace = True)
# for i in range(len(res['user_id'])):
#     ym = np.array([res['user_buy_cnt_05'][i],res['user_buy_cnt_06'][i],res['user_buy_cnt_07'][i],res['user_buy_cnt_08'][i],res['user_buy_cnt_09'][i],res['user_buy_cnt_10'][i],res['user_buy_cnt_11'][i]])
#     yy = pd.DataFrame({"k2": ym})
#     # print(yy)
#     temptemp = pd.concat([xx,yy],axis=1)
#     # print(temp)
#     est = smf.ols(formula='yy ~ xx', data=temptemp).fit()
#     res['slope'][i] = est.params['xx']
#     # resu = smf.ols(y=yy, x=xx) 
#     # print(est.params['xx'])
#res


In [14]:
# del res['cnt_09'],res['cnt_08'],res['cnt_07'],res['cnt_06'],res['cnt_05']

In [17]:
temptemp = pd.read_csv('res.csv')
temptemp = temptemp[['user_id','slope']]
temptemp['user_id'] = temptemp['user_id'].astype(str)
matrix = matrix.merge(res, on='user_id', how='left').merge(temp,on='user_id',how='left').merge(temptemp,on='user_id',how='left')
del res,temptemp
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,user_buy_cnt_08,user_buy_cnt_07,user_buy_cnt_06,user_buy_cnt_05,user_buy_mean_between_months,user_buy_std_between_months,user_buy_min_between_months,user_buy_max_between_months,user_buy_sum_between_months,slope
0,34176,3906,0,train,451,256,45,109,108,174,...,1.0,1.0,4.0,5.0,4.857143,4.879500,1.0,15.0,34.0,1.250000
1,34176,121,0,train,451,256,45,109,108,174,...,1.0,1.0,4.0,5.0,4.857143,4.879500,1.0,15.0,34.0,1.250000
2,34176,4356,1,train,451,256,45,109,108,174,...,1.0,1.0,4.0,5.0,4.857143,4.879500,1.0,15.0,34.0,1.250000
3,34176,2217,0,train,451,256,45,109,108,174,...,1.0,1.0,4.0,5.0,4.857143,4.879500,1.0,15.0,34.0,1.250000
4,230784,4818,0,train,54,31,17,20,19,163,...,0.0,1.0,4.0,0.0,1.000000,1.414214,0.0,4.0,7.0,-0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,2.0,1.0,4.0,3.0,3.714286,2.288689,1.0,8.0,26.0,0.428571
522337,97919,2341,-1,test,55,29,14,17,17,138,...,0.0,2.0,1.0,0.0,1.142857,1.864454,0.0,5.0,8.0,0.392857
522338,97919,3971,-1,test,55,29,14,17,17,138,...,0.0,2.0,1.0,0.0,1.142857,1.864454,0.0,5.0,8.0,0.392857
522339,32639,3536,-1,test,72,46,24,33,35,172,...,1.0,1.0,0.0,1.0,1.142857,0.690066,0.0,2.0,8.0,0.250000


In [18]:
# action ratio
matrix = matrix.fillna(0)
matrix['user_buy_cnt_11_ratio'] = matrix['user_buy_cnt_11'] / matrix['u9']
matrix['user_buy_cnt_10_ratio'] = matrix['user_buy_cnt_10'] / matrix['u9']
matrix['user_buy_cnt_09_ratio'] = matrix['user_buy_cnt_09'] / matrix['u9']
matrix['user_buy_cnt_08_ratio'] = matrix['user_buy_cnt_08'] / matrix['u9']
matrix['user_buy_cnt_07_ratio'] = matrix['user_buy_cnt_07'] / matrix['u9']
matrix['user_buy_cnt_06_ratio'] = matrix['user_buy_cnt_06'] / matrix['u9']
matrix['user_buy_cnt_05_ratio'] = matrix['user_buy_cnt_05'] / matrix['u9']
matrix = matrix.fillna(0)
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,user_buy_max_between_months,user_buy_sum_between_months,slope,user_buy_cnt_11_ratio,user_buy_cnt_10_ratio,user_buy_cnt_09_ratio,user_buy_cnt_08_ratio,user_buy_cnt_07_ratio,user_buy_cnt_06_ratio,user_buy_cnt_05_ratio
0,34176,3906,0,train,451,256,45,109,108,174,...,15.0,34.0,1.250000,0.441176,0.176471,0.058824,0.029412,0.029412,0.117647,0.147059
1,34176,121,0,train,451,256,45,109,108,174,...,15.0,34.0,1.250000,0.441176,0.176471,0.058824,0.029412,0.029412,0.117647,0.147059
2,34176,4356,1,train,451,256,45,109,108,174,...,15.0,34.0,1.250000,0.441176,0.176471,0.058824,0.029412,0.029412,0.117647,0.147059
3,34176,2217,0,train,451,256,45,109,108,174,...,15.0,34.0,1.250000,0.441176,0.176471,0.058824,0.029412,0.029412,0.117647,0.147059
4,230784,4818,0,train,54,31,17,20,19,163,...,4.0,7.0,-0.142857,0.142857,0.142857,0.000000,0.000000,0.142857,0.571429,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,8.0,26.0,0.428571,0.115385,0.307692,0.192308,0.076923,0.038462,0.153846,0.115385
522337,97919,2341,-1,test,55,29,14,17,17,138,...,5.0,8.0,0.392857,0.625000,0.000000,0.000000,0.000000,0.250000,0.125000,0.000000
522338,97919,3971,-1,test,55,29,14,17,17,138,...,5.0,8.0,0.392857,0.625000,0.000000,0.000000,0.000000,0.250000,0.125000,0.000000
522339,32639,3536,-1,test,72,46,24,33,35,172,...,2.0,8.0,0.250000,0.250000,0.250000,0.125000,0.125000,0.125000,0.000000,0.125000


In [19]:
mer_actions_at_nov = buy_actions_at_nov.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'m_cnt_11': 'count'}).fillna(0)
del mer_actions_at_nov['month']
mer_actions_at_oct = buy_actions_at_oct.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'m_cnt_10': 'count'}).fillna(0)
del mer_actions_at_oct['month']
mer_actions_at_sep = buy_actions_at_sep.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'m_cnt_09': 'count'}).fillna(0)
del mer_actions_at_sep['month']
mer_actions_at_aug = buy_actions_at_aug.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'m_cnt_08': 'count'}).fillna(0)
del mer_actions_at_aug['month']
mer_actions_at_jul = buy_actions_at_jul.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'m_cnt_07': 'count'}).fillna(0)
del mer_actions_at_jul['month']
mer_actions_at_jun = buy_actions_at_jun.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'m_cnt_06': 'count'}).fillna(0)
del mer_actions_at_jun['month']
mer_actions_at_may = buy_actions_at_may.groupby(['merchant_id', 'month'], as_index=False)['month'].agg({'m_cnt_05': 'count'}).fillna(0)
del mer_actions_at_may['month']

In [20]:
res1 = pd.merge(mer_actions_at_nov,mer_actions_at_oct, on=['merchant_id'], how='left')
res1 = res1.merge(mer_actions_at_sep, on=['merchant_id'], how='left')
res1 = res1.merge(mer_actions_at_aug, on=['merchant_id'], how='left')
res1 = res1.merge(mer_actions_at_jul, on=['merchant_id'], how='left')
res1 = res1.merge(mer_actions_at_jun, on=['merchant_id'], how='left')
res1 = res1.merge(mer_actions_at_may, on=['merchant_id'], how='left')
res1 = res1.fillna(0)
# res['user_buy_deviation_between_months'] = abs(res['cnt'] - res['mean_of_num_of_actions_not_in_nov'])
temp=[] 
temp=pd.DataFrame(temp)
temp['merchant_id'] = res1['merchant_id']
temp['merchant_buy_mean_between_months'] = res1.mean(axis=1)
temp['merchant_buy_std_between_months'] = res1.std(axis=1)
temp['merchant_buy_min_between_months'] = res1.min(axis=1)
temp['merchant_buy_max_between_months'] = res1.max(axis=1)
temp['merchant_buy_sum_between_months'] = res1.sum(axis=1)
temp
# res = res.loc[:, ['user_id', 'merchant_id', 'user_buy_deviation_between_nov_and_before']]

,merchant_id,merchant_buy_mean_between_months,merchant_buy_std_between_months,merchant_buy_min_between_months,merchant_buy_max_between_months,merchant_buy_sum_between_months
0,1,2529.285714,2268.896921,1320.0,7594.0,17705.0
1,10,161.857143,368.508900,1.0,997.0,1133.0
2,100,76.857143,109.929415,0.0,251.0,538.0
3,1000,137.000000,182.047613,27.0,544.0,959.0
4,1001,28.000000,21.931712,8.0,68.0,196.0
...,...,...,...,...,...,...
4987,995,34.857143,30.295371,6.0,89.0,244.0
4988,996,17.428571,40.057696,0.0,108.0,122.0
4989,997,39.428571,48.822321,3.0,126.0,276.0
4990,998,97.714286,104.172476,0.0,261.0,684.0


In [22]:
import statsmodels.formula.api as smf
xm = np.array([1,2,3,4,5,6,7])
xx = pd.DataFrame({"k1": xm})
res1['slope_m'] = None
# res['slope'].fillna(0,inplace = True)
for i in range(len(res1['merchant_id'])):
    ym = np.array([res1['m_cnt_05'][i],res1['m_cnt_06'][i],res1['m_cnt_07'][i],res1['m_cnt_08'][i],res1['m_cnt_09'][i],res1['m_cnt_10'][i],res1['m_cnt_11'][i]])
    yy = pd.DataFrame({"k2": ym})
    # print(yy)
    temptemp = pd.concat([xx,yy],axis=1)
    # print(temp)
    est = smf.ols(formula='yy ~ xx', data=temptemp).fit()
    res1['slope_m'][i] = est.params['xx']
    # resu = smf.ols(y=yy, x=xx) 
    # print(est.params['xx'])
res1

,merchant_id,m_cnt_11,m_cnt_10,m_cnt_09,m_cnt_08,m_cnt_07,m_cnt_06,m_cnt_05,slope_m
0,1,7594,1558.0,2143.0,1353.0,1320.0,1408.0,2329.0,604.214
1,10,997,45.0,29.0,14.0,1.0,24.0,23.0,106.857
2,100,251,215.0,68.0,1.0,2.0,1.0,0.0,44.5357
3,1000,544,84.0,83.0,111.0,80.0,30.0,27.0,59.3571
4,1001,44,12.0,15.0,8.0,15.0,34.0,68.0,-4.14286
...,...,...,...,...,...,...,...,...,...
4987,995,89,60.0,18.0,15.0,6.0,41.0,15.0,9.71429
4988,996,108,9.0,2.0,2.0,0.0,0.0,1.0,12.1786
4989,997,91,24.0,126.0,17.0,3.0,3.0,12.0,14.3571
4990,998,107,261.0,216.0,69.0,31.0,0.0,0.0,36.7143


In [23]:
# del res1['cnt_09'],res1['cnt_08'],res1['cnt_07'],res1['cnt_06'],res1['cnt_05']
matrix = matrix.merge(res1, on='merchant_id', how='left').merge(temp,on='merchant_id',how='left')
del res1
# action ratio
matrix = matrix.fillna(0)
matrix['mer_buy_cnt_11_ratio'] = matrix['m_cnt_11'] / matrix['m8']
matrix['mer_buy_cnt_10_ratio'] = matrix['m_cnt_10'] / matrix['m8']
matrix['mer_buy_cnt_09_ratio'] = matrix['m_cnt_09'] / matrix['m8']
matrix['mer_buy_cnt_08_ratio'] = matrix['m_cnt_08'] / matrix['m8']
matrix['mer_buy_cnt_07_ratio'] = matrix['m_cnt_07'] / matrix['m8']
matrix['mer_buy_cnt_06_ratio'] = matrix['m_cnt_06'] / matrix['m8']
matrix['mer_buy_cnt_05_ratio'] = matrix['m_cnt_05'] / matrix['m8']
matrix = matrix.fillna(0)
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,merchant_buy_min_between_months,merchant_buy_max_between_months,merchant_buy_sum_between_months,mer_buy_cnt_11_ratio,mer_buy_cnt_10_ratio,mer_buy_cnt_09_ratio,mer_buy_cnt_08_ratio,mer_buy_cnt_07_ratio,mer_buy_cnt_06_ratio,mer_buy_cnt_05_ratio
0,34176,3906,0,train,451,256,45,109,108,174,...,14.0,178.0,410.0,0.434146,0.070732,0.148780,0.039024,0.034146,0.151220,0.121951
1,34176,121,0,train,451,256,45,109,108,174,...,77.0,3299.0,4780.0,0.690167,0.055858,0.093933,0.034310,0.062552,0.016109,0.047071
2,34176,4356,1,train,451,256,45,109,108,174,...,0.0,391.0,963.0,0.173416,0.111111,0.127726,0.154725,0.406023,0.026999,0.000000
3,34176,2217,0,train,451,256,45,109,108,174,...,81.0,2715.0,3721.0,0.729643,0.099704,0.045418,0.021768,0.034668,0.041924,0.026874
4,230784,4818,0,train,54,31,17,20,19,163,...,68.0,2161.0,2733.0,0.790706,0.037322,0.031467,0.024881,0.032565,0.047567,0.035492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,66.0,191.0,687.0,0.278020,0.141194,0.129549,0.098981,0.128093,0.128093,0.096070
522337,97919,2341,-1,test,55,29,14,17,17,138,...,33.0,309.0,815.0,0.379141,0.106748,0.040491,0.109202,0.139877,0.136196,0.088344
522338,97919,3971,-1,test,55,29,14,17,17,138,...,54.0,1767.0,2608.0,0.677531,0.067868,0.084739,0.046396,0.049463,0.053298,0.020706
522339,32639,3536,-1,test,72,46,24,33,35,172,...,0.0,753.0,793.0,0.949559,0.026482,0.005044,0.016393,0.001261,0.001261,0.000000


In [24]:
um_actions_at_nov = buy_actions_at_nov.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'um_cnt_11': 'count'}).fillna(0)
del um_actions_at_nov['month']
um_actions_at_oct = buy_actions_at_oct.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'um_cnt_10': 'count'}).fillna(0)
del um_actions_at_oct['month']
um_actions_at_sep = buy_actions_at_sep.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'um_cnt_09': 'count'}).fillna(0)
del um_actions_at_sep['month']
um_actions_at_aug = buy_actions_at_aug.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'um_cnt_08': 'count'}).fillna(0)
del um_actions_at_aug['month']
um_actions_at_jul = buy_actions_at_jul.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'um_cnt_07': 'count'}).fillna(0)
del um_actions_at_jul['month']
um_actions_at_jun = buy_actions_at_jun.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'um_cnt_06': 'count'}).fillna(0)
del um_actions_at_jun['month']
um_actions_at_may = buy_actions_at_may.groupby(['user_id','merchant_id', 'month'], as_index=False)['month'].agg({'um_cnt_05': 'count'}).fillna(0)
del um_actions_at_may['month']


In [25]:
res2 = pd.merge(um_actions_at_nov,um_actions_at_oct, on=['user_id','merchant_id'], how='left')
res2 = res2.merge(um_actions_at_sep, on=['user_id','merchant_id'], how='left')
res2 = res2.merge(um_actions_at_aug, on=['user_id','merchant_id'], how='left')
res2 = res2.merge(um_actions_at_jul, on=['user_id','merchant_id'], how='left')
res2 = res2.merge(um_actions_at_jun, on=['user_id','merchant_id'], how='left')
res2 = res2.merge(um_actions_at_may, on=['user_id','merchant_id'], how='left')
# res['user_buy_deviation_between_months'] = abs(res['cnt'] - res['mean_of_num_of_actions_not_in_nov'])
res2 = res2.fillna(0)
temp=[] 
temp=pd.DataFrame(temp)
temp['user_id'] = res2['user_id']
temp['merchant_id'] = res2['merchant_id']
temp['um_buy_mean_between_months'] = res2.mean(axis=1)
temp['um_buy_std_between_months'] = res2.std(axis=1)
temp['um_buy_min_between_months'] = res2.min(axis=1)
temp['um_buy_max_between_months'] = res2.max(axis=1)
temp['um_buy_sum_between_months'] = res2.sum(axis=1)
temp.head(5)

,user_id,merchant_id,um_buy_mean_between_months,um_buy_std_between_months,um_buy_min_between_months,um_buy_max_between_months,um_buy_sum_between_months
0,1,1019,0.571429,1.511858,0.0,4.0,4.0
1,10,938,0.285714,0.755929,0.0,2.0,2.0
2,100,1,0.571429,1.133893,0.0,3.0,4.0
3,100,1461,0.142857,0.377964,0.0,1.0,1.0
4,100,2537,0.285714,0.755929,0.0,2.0,2.0


In [27]:
# import statsmodels.formula.api as smf
xm = np.array([1,2,3,4,5,6,7])
xx = pd.DataFrame({"k1": xm})
res2['slope_um'] = None
# res['slope'].fillna(0,inplace = True)
for i in range(len(res2['user_id'])):
    ym = np.array([res2['um_cnt_05'][i],res2['um_cnt_06'][i],res2['um_cnt_07'][i],res2['um_cnt_08'][i],res2['um_cnt_09'][i],res2['um_cnt_10'][i],res2['um_cnt_11'][i]])
    yy = pd.DataFrame({"k2": ym})
    # print(yy)
    temptemp = pd.concat([xx,yy],axis=1)
    # print(temp)
    est = smf.ols(formula='yy ~ xx', data=temptemp).fit()
    res2['slope_um'][i] = est.params['xx']
    # resu = smf.ols(y=yy, x=xx) 
    # print(est.params['xx'])
res2

,user_id,merchant_id,um_cnt_11,um_cnt_10,um_cnt_09,um_cnt_08,um_cnt_07,um_cnt_06,um_cnt_05,slope_um
0,1,1019,4,0.0,0.0,0.0,0.0,0.0,0.0,0.428571
1,10,938,2,0.0,0.0,0.0,0.0,0.0,0.0,0.214286
2,100,1,1,3.0,0.0,0.0,0.0,0.0,0.0,0.321429
3,100,1461,1,0.0,0.0,0.0,0.0,0.0,0.0,0.107143
4,100,2537,2,0.0,0.0,0.0,0.0,0.0,0.0,0.214286
...,...,...,...,...,...,...,...,...,...,...
981657,99998,1131,1,0.0,0.0,0.0,0.0,0.0,0.0,0.107143
981658,99998,4514,1,0.0,0.0,0.0,0.0,0.0,0.0,0.107143
981659,99998,798,4,0.0,0.0,0.0,0.0,0.0,0.0,0.428571
981660,99999,141,1,0.0,0.0,0.0,0.0,0.0,0.0,0.107143


In [28]:
# del res2['cnt_09'],res2['cnt_08'],res2['cnt_07'],res2['cnt_06'],res2['cnt_05']
matrix = matrix.merge(res2, on=['user_id','merchant_id'], how='left').merge(temp,on=['user_id','merchant_id'],how='left')
del res2
# action ratio
matrix = matrix.fillna(0)
matrix['um_buy_cnt_11_ratio'] = matrix['um_cnt_11'] / matrix['um7']
matrix['um_buy_cnt_10_ratio'] = matrix['um_cnt_10'] / matrix['um7']
matrix['um_buy_cnt_09_ratio'] = matrix['um_cnt_09'] / matrix['um7']
matrix['um_buy_cnt_08_ratio'] = matrix['um_cnt_08'] / matrix['um7']
matrix['um_buy_cnt_07_ratio'] = matrix['um_cnt_07'] / matrix['um7']
matrix['um_buy_cnt_06_ratio'] = matrix['um_cnt_06'] / matrix['um7']
matrix['um_buy_cnt_05_ratio'] = matrix['um_cnt_05'] / matrix['um7']
matrix = matrix.fillna(0)
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_buy_min_between_months,um_buy_max_between_months,um_buy_sum_between_months,um_buy_cnt_11_ratio,um_buy_cnt_10_ratio,um_buy_cnt_09_ratio,um_buy_cnt_08_ratio,um_buy_cnt_07_ratio,um_buy_cnt_06_ratio,um_buy_cnt_05_ratio
0,34176,3906,0,train,451,256,45,109,108,174,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,34176,121,0,train,451,256,45,109,108,174,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34176,4356,1,train,451,256,45,109,108,174,...,0.0,6.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34176,2217,0,train,451,256,45,109,108,174,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,230784,4818,0,train,54,31,17,20,19,163,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
522337,97919,2341,-1,test,55,29,14,17,17,138,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
522338,97919,3971,-1,test,55,29,14,17,17,138,...,0.0,4.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
522339,32639,3536,-1,test,72,46,24,33,35,172,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# Repeat buyer features
buy_actions = user_log[user_log['action_type'] == 2]
um_groups = buy_actions.groupby(['merchant_id','user_id'])
temp = um_groups.size().reset_index().rename(columns={0:'mu1'}) #统计行为个数
temp = temp[temp['mu1']>=2]
temp = temp.groupby('merchant_id').size().reset_index().rename(columns={0:'repeat_buyer_count'})
temp

,merchant_id,repeat_buyer_count
0,1,3779
1,10,107
2,100,127
3,1000,76
4,1001,32
...,...,...
4970,995,15
4971,996,15
4972,997,28
4973,998,164


In [30]:
# Repeat buyer features
matrix = matrix.merge(temp, on=['merchant_id'], how='left')
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_buy_max_between_months,um_buy_sum_between_months,um_buy_cnt_11_ratio,um_buy_cnt_10_ratio,um_buy_cnt_09_ratio,um_buy_cnt_08_ratio,um_buy_cnt_07_ratio,um_buy_cnt_06_ratio,um_buy_cnt_05_ratio,repeat_buyer_count
0,34176,3906,0,train,451,256,45,109,108,174,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0
1,34176,121,0,train,451,256,45,109,108,174,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1001.0
2,34176,4356,1,train,451,256,45,109,108,174,...,6.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,170.0
3,34176,2217,0,train,451,256,45,109,108,174,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,316.0
4,230784,4818,0,train,54,31,17,20,19,163,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,543.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0
522337,97919,2341,-1,test,55,29,14,17,17,138,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,175.0
522338,97919,3971,-1,test,55,29,14,17,17,138,...,4.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,431.0
522339,32639,3536,-1,test,72,46,24,33,35,172,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0


In [31]:
# 最后一个月 一个星期 一天的购物数量
actions_at_1111 = user_log[user_log['time_stamp'] == "1900-11-11"]
actions_at_lastweek = user_log[user_log['time_stamp'] > '1900-11-04']
actions_at_lastweek

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type,month
171,328862,406349,1280,2700,5476.0,1900-11-11,0,11
172,328862,406349,1280,2700,5476.0,1900-11-11,0,11
173,328862,807126,1181,1963,6109.0,1900-11-11,0,11
174,328862,406349,1280,2700,5476.0,1900-11-11,2,11
175,328862,406349,1280,2700,5476.0,1900-11-11,0,11
...,...,...,...,...,...,...,...,...
54925325,208016,107662,898,1346,7995.0,1900-11-10,0,11
54925326,208016,1058313,898,1346,7995.0,1900-11-10,0,11
54925327,208016,449814,898,983,7995.0,1900-11-10,0,11
54925328,208016,634856,898,1346,7995.0,1900-11-10,0,11


In [32]:
# 最后一个星期 一天的购物数量
u_buy_actions_at_1111 = actions_at_1111.groupby(['user_id']).size().reset_index().rename(columns={0:'u1_buy_1111'})
matrix = matrix.merge(u_buy_actions_at_1111, on=['user_id'], how='left')
del u_buy_actions_at_1111
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_buy_sum_between_months,um_buy_cnt_11_ratio,um_buy_cnt_10_ratio,um_buy_cnt_09_ratio,um_buy_cnt_08_ratio,um_buy_cnt_07_ratio,um_buy_cnt_06_ratio,um_buy_cnt_05_ratio,repeat_buyer_count,u1_buy_1111
0,34176,3906,0,train,451,256,45,109,108,174,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,36
1,34176,121,0,train,451,256,45,109,108,174,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1001.0,36
2,34176,4356,1,train,451,256,45,109,108,174,...,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,170.0,36
3,34176,2217,0,train,451,256,45,109,108,174,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,316.0,36
4,230784,4818,0,train,54,31,17,20,19,163,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,543.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,17
522337,97919,2341,-1,test,55,29,14,17,17,138,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,175.0,9
522338,97919,3971,-1,test,55,29,14,17,17,138,...,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,431.0,9
522339,32639,3536,-1,test,72,46,24,33,35,172,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0,30


In [33]:
# Latest one-week features and latest one-month features
u_buy_actions_at_lastweek = actions_at_lastweek.groupby(['user_id']).size().reset_index().rename(columns={0:'u1_buy_last_week'})
# u_buy_actions_at_lastweek
matrix = matrix.merge(u_buy_actions_at_lastweek, on=['user_id'], how='left')
del u_buy_actions_at_lastweek
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_buy_cnt_11_ratio,um_buy_cnt_10_ratio,um_buy_cnt_09_ratio,um_buy_cnt_08_ratio,um_buy_cnt_07_ratio,um_buy_cnt_06_ratio,um_buy_cnt_05_ratio,repeat_buyer_count,u1_buy_1111,u1_buy_last_week
0,34176,3906,0,train,451,256,45,109,108,174,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,36,119
1,34176,121,0,train,451,256,45,109,108,174,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1001.0,36,119
2,34176,4356,1,train,451,256,45,109,108,174,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,170.0,36,119
3,34176,2217,0,train,451,256,45,109,108,174,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,316.0,36,119
4,230784,4818,0,train,54,31,17,20,19,163,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,543.0,2,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,17,115
522337,97919,2341,-1,test,55,29,14,17,17,138,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,175.0,9,30
522338,97919,3971,-1,test,55,29,14,17,17,138,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,431.0,9,30
522339,32639,3536,-1,test,72,46,24,33,35,172,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,89.0,30,34


In [34]:
matrix = matrix.fillna(0)
matrix['user_buy_cnt_1111_ratio'] = matrix['u1_buy_1111'] / matrix['u9']
matrix['user_buy_cnt_last_week_ratio'] = matrix['u1_buy_last_week'] / matrix['u9']
matrix = matrix.fillna(0)
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_buy_cnt_09_ratio,um_buy_cnt_08_ratio,um_buy_cnt_07_ratio,um_buy_cnt_06_ratio,um_buy_cnt_05_ratio,repeat_buyer_count,u1_buy_1111,u1_buy_last_week,user_buy_cnt_1111_ratio,user_buy_cnt_last_week_ratio
0,34176,3906,0,train,451,256,45,109,108,174,...,0.0,0.0,0.0,0.0,0.0,66.0,36,119,1.058824,3.500000
1,34176,121,0,train,451,256,45,109,108,174,...,0.0,0.0,0.0,0.0,0.0,1001.0,36,119,1.058824,3.500000
2,34176,4356,1,train,451,256,45,109,108,174,...,0.0,0.0,0.0,0.0,0.0,170.0,36,119,1.058824,3.500000
3,34176,2217,0,train,451,256,45,109,108,174,...,0.0,0.0,0.0,0.0,0.0,316.0,36,119,1.058824,3.500000
4,230784,4818,0,train,54,31,17,20,19,163,...,0.0,0.0,0.0,0.0,0.0,543.0,2,12,0.285714,1.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,0.0,0.0,0.0,0.0,0.0,72.0,17,115,0.653846,4.423077
522337,97919,2341,-1,test,55,29,14,17,17,138,...,0.0,0.0,0.0,0.0,0.0,175.0,9,30,1.125000,3.750000
522338,97919,3971,-1,test,55,29,14,17,17,138,...,0.0,0.0,0.0,0.0,0.0,431.0,9,30,1.125000,3.750000
522339,32639,3536,-1,test,72,46,24,33,35,172,...,0.0,0.0,0.0,0.0,0.0,89.0,30,34,3.750000,4.250000


In [35]:
# User aggregation features
um_buy_actions_day = buy_actions.groupby(['merchant_id', 'user_id'], as_index=False)['time_stamp'].agg({'um_agg_day': 'count'}).fillna(0)
um_buy_actions_day.head(5)

,merchant_id,user_id,um_agg_day
0,1,100,4
1,1,100099,2
2,1,100136,1
3,1,100196,13
4,1,100257,6


In [36]:
um_buy_actions_day_tmp = um_buy_actions_day.groupby('merchant_id')['um_agg_day'].agg([('um_agg_day_min', 'min'), ('um_agg_day_max', 'max'),('um_agg_day_mean', 'mean'),('um_agg_day_std', 'std'), ('um_agg_day_sum', 'sum')]).reset_index().fillna(0)
um_buy_actions_day_tmp.head(5)
matrix = matrix.merge(um_buy_actions_day_tmp, on=['merchant_id'], how='left')
del um_buy_actions_day,um_buy_actions_day_tmp
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,repeat_buyer_count,u1_buy_1111,u1_buy_last_week,user_buy_cnt_1111_ratio,user_buy_cnt_last_week_ratio,um_agg_day_min,um_agg_day_max,um_agg_day_mean,um_agg_day_std,um_agg_day_sum
0,34176,3906,0,train,451,256,45,109,108,174,...,66.0,36,119,1.058824,3.500000,1,8,1.322581,0.791831,410
1,34176,121,0,train,451,256,45,109,108,174,...,1001.0,36,119,1.058824,3.500000,1,16,1.735657,1.422353,4780
2,34176,4356,1,train,451,256,45,109,108,174,...,170.0,36,119,1.058824,3.500000,1,11,1.426667,1.030053,963
3,34176,2217,0,train,451,256,45,109,108,174,...,316.0,36,119,1.058824,3.500000,1,17,1.123151,0.502937,3721
4,230784,4818,0,train,54,31,17,20,19,163,...,543.0,2,12,0.285714,1.714286,1,15,1.739656,1.438444,2733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,72.0,17,115,0.653846,4.423077,1,7,1.201049,0.658874,687
522337,97919,2341,-1,test,55,29,14,17,17,138,...,175.0,9,30,1.125000,3.750000,1,17,2.089744,2.029258,815
522338,97919,3971,-1,test,55,29,14,17,17,138,...,431.0,9,30,1.125000,3.750000,1,10,1.306613,0.715161,2608
522339,32639,3536,-1,test,72,46,24,33,35,172,...,89.0,30,34,3.750000,4.250000,1,9,1.199697,0.650920,793


In [37]:
# User aggregation features
um_buy_actions_brand = buy_actions.groupby(['merchant_id', 'user_id'], as_index=False)['brand_id'].agg({'um_agg_brand': 'nunique'}).fillna(0)
um_buy_actions_brand.head(5)

,merchant_id,user_id,um_agg_brand
0,1,100,1
1,1,100099,1
2,1,100136,1
3,1,100196,2
4,1,100257,1


In [38]:
um_buy_actions_brand_tmp = um_buy_actions_brand.groupby('merchant_id')['um_agg_brand'].agg([('um_agg_brand_min', 'min'), ('um_agg_brand_max', 'max'),('um_agg_brand_mean', 'mean'),('um_agg_brand_std', 'std'), ('um_agg_brand_sum', 'sum')]).reset_index().fillna(0)
print(um_buy_actions_brand_tmp.head(5))

matrix = matrix.merge(um_buy_actions_brand_tmp, on=['merchant_id'], how='left')
del um_buy_actions_brand,um_buy_actions_brand_tmp
matrix

  merchant_id  um_agg_brand_min  um_agg_brand_max  um_agg_brand_mean  \
0           1                 1                 2           1.020089   
1          10                 1                 1           1.000000   
2         100                 1                 2           1.049231   
3        1000                 1                 2           1.016279   
4        1001                 1                 1           1.000000   

   um_agg_brand_std  um_agg_brand_sum  
0          0.140313              7820  
1          0.000000               995  
2          0.216683               341  
3          0.126620               874  
4          0.000000               158  


,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_agg_day_min,um_agg_day_max,um_agg_day_mean,um_agg_day_std,um_agg_day_sum,um_agg_brand_min,um_agg_brand_max,um_agg_brand_mean,um_agg_brand_std,um_agg_brand_sum
0,34176,3906,0,train,451,256,45,109,108,174,...,1,8,1.322581,0.791831,410,1,1,1.000000,0.000000,310
1,34176,121,0,train,451,256,45,109,108,174,...,1,16,1.735657,1.422353,4780,1,1,1.000000,0.000000,2754
2,34176,4356,1,train,451,256,45,109,108,174,...,1,11,1.426667,1.030053,963,1,1,1.000000,0.000000,675
3,34176,2217,0,train,451,256,45,109,108,174,...,1,17,1.123151,0.502937,3721,1,1,1.000000,0.000000,3313
4,230784,4818,0,train,54,31,17,20,19,163,...,1,15,1.739656,1.438444,2733,1,1,1.000000,0.000000,1571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,1,7,1.201049,0.658874,687,1,2,1.026224,0.159940,587
522337,97919,2341,-1,test,55,29,14,17,17,138,...,1,17,2.089744,2.029258,815,1,7,1.412821,0.769829,551
522338,97919,3971,-1,test,55,29,14,17,17,138,...,1,10,1.306613,0.715161,2608,1,1,1.000000,0.000000,1996
522339,32639,3536,-1,test,72,46,24,33,35,172,...,1,9,1.199697,0.650920,793,1,2,1.003026,0.054965,663


In [39]:
# User aggregation features
um_buy_actions_item = buy_actions.groupby(['merchant_id', 'user_id'], as_index=False)['item_id'].agg({'um_agg_item': 'nunique'}).fillna(0)
um_buy_actions_item.head(5)

,merchant_id,user_id,um_agg_item
0,1,100,4
1,1,100099,2
2,1,100136,1
3,1,100196,13
4,1,100257,6


In [40]:
um_buy_actions_item_tmp = um_buy_actions_item.groupby('merchant_id')['um_agg_item'].agg([('um_agg_item_min', 'min'), ('um_agg_item_max', 'max'),('um_agg_item_mean', 'mean'),('um_agg_item_std', 'std'),('um_agg_item_sum', 'sum')]).reset_index().fillna(0)
um_buy_actions_item_tmp.head(5)

matrix = matrix.merge(um_buy_actions_item_tmp, on=['merchant_id'], how='left')
del um_buy_actions_item,um_buy_actions_item_tmp
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_agg_brand_min,um_agg_brand_max,um_agg_brand_mean,um_agg_brand_std,um_agg_brand_sum,um_agg_item_min,um_agg_item_max,um_agg_item_mean,um_agg_item_std,um_agg_item_sum
0,34176,3906,0,train,451,256,45,109,108,174,...,1,1,1.000000,0.000000,310,1,6,1.258065,0.676357,390
1,34176,121,0,train,451,256,45,109,108,174,...,1,1,1.000000,0.000000,2754,1,13,1.633987,1.247600,4500
2,34176,4356,1,train,451,256,45,109,108,174,...,1,1,1.000000,0.000000,675,1,6,1.149630,0.483998,776
3,34176,2217,0,train,451,256,45,109,108,174,...,1,1,1.000000,0.000000,3313,1,9,1.074857,0.354166,3561
4,230784,4818,0,train,54,31,17,20,19,163,...,1,1,1.000000,0.000000,1571,1,15,1.661999,1.304944,2611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,1,2,1.026224,0.159940,587,1,5,1.148601,0.470438,657
522337,97919,2341,-1,test,55,29,14,17,17,138,...,1,7,1.412821,0.769829,551,1,15,1.892308,1.742761,738
522338,97919,3971,-1,test,55,29,14,17,17,138,...,1,1,1.000000,0.000000,1996,1,9,1.276553,0.661639,2548
522339,32639,3536,-1,test,72,46,24,33,35,172,...,1,2,1.003026,0.054965,663,1,5,1.122542,0.417558,742


In [41]:
# Merchant aggregation features-day
mu_buy_actions_day = buy_actions.groupby(['user_id','merchant_id'], as_index=False)['time_stamp'].agg({'mu_agg_day': 'count'}).fillna(0)
print(mu_buy_actions_day.head(5))
mu_buy_actions_day_tmp = mu_buy_actions_day.groupby('user_id')['mu_agg_day'].agg([('mu_agg_day_min', 'min'), ('mu_agg_day_max', 'max'),('mu_agg_day_mean', 'mean'),('mu_agg_day_std', 'std'), ('mu_agg_day_sum', 'sum')]).reset_index().fillna(0)
mu_buy_actions_day_tmp.head(5)

  user_id merchant_id  mu_agg_day
0       1        1019           4
1       1        4026           1
2       1         925           1
3      10        1364           5
4      10         938           2


,user_id,mu_agg_day_min,mu_agg_day_max,mu_agg_day_mean,mu_agg_day_std,mu_agg_day_sum
0,1,1,4,2.000000,1.732051,6
1,10,2,5,3.500000,2.121320,7
2,100,1,4,1.666667,1.211060,10
3,1000,1,3,1.400000,0.894427,7
4,10000,1,2,1.307692,0.480384,17


In [42]:
matrix = matrix.merge(mu_buy_actions_day_tmp, on=['user_id'], how='left')
del mu_buy_actions_day_tmp,mu_buy_actions_day
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,um_agg_item_min,um_agg_item_max,um_agg_item_mean,um_agg_item_std,um_agg_item_sum,mu_agg_day_min,mu_agg_day_max,mu_agg_day_mean,mu_agg_day_std,mu_agg_day_sum
0,34176,3906,0,train,451,256,45,109,108,174,...,1,6,1.258065,0.676357,390,1,6,1.545455,1.299350,34
1,34176,121,0,train,451,256,45,109,108,174,...,1,13,1.633987,1.247600,4500,1,6,1.545455,1.299350,34
2,34176,4356,1,train,451,256,45,109,108,174,...,1,6,1.149630,0.483998,776,1,6,1.545455,1.299350,34
3,34176,2217,0,train,451,256,45,109,108,174,...,1,9,1.074857,0.354166,3561,1,6,1.545455,1.299350,34
4,230784,4818,0,train,54,31,17,20,19,163,...,1,15,1.661999,1.304944,2611,1,2,1.166667,0.408248,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,1,5,1.148601,0.470438,657,1,2,1.238095,0.436436,26
522337,97919,2341,-1,test,55,29,14,17,17,138,...,1,15,1.892308,1.742761,738,1,4,2.000000,1.414214,8
522338,97919,3971,-1,test,55,29,14,17,17,138,...,1,9,1.276553,0.661639,2548,1,4,2.000000,1.414214,8
522339,32639,3536,-1,test,72,46,24,33,35,172,...,1,5,1.122542,0.417558,742,1,1,1.000000,0.000000,8


In [43]:
# Merchant aggregation features-day
mu_buy_actions_brand = buy_actions.groupby(['user_id','merchant_id'], as_index=False)['brand_id'].agg({'mu_agg_brand': 'nunique'}).fillna(0)
print(mu_buy_actions_brand.head(5))
mu_buy_actions_brand_tmp = mu_buy_actions_brand.groupby('user_id')['mu_agg_brand'].agg([('mu_agg_brand_min', 'min'), ('mu_agg_brand_max', 'max'),('mu_agg_brand_mean', 'mean'),('mu_agg_brand_std', 'std'), ('mu_agg_brand_sum', 'sum')]).reset_index().fillna(0)
mu_buy_actions_brand_tmp.head(5)

  user_id merchant_id  mu_agg_brand
0       1        1019             1
1       1        4026             1
2       1         925             1
3      10        1364             1
4      10         938             1


,user_id,mu_agg_brand_min,mu_agg_brand_max,mu_agg_brand_mean,mu_agg_brand_std,mu_agg_brand_sum
0,1,1,1,1.0,0.0,3
1,10,1,1,1.0,0.0,2
2,100,1,1,1.0,0.0,6
3,1000,1,1,1.0,0.0,5
4,10000,1,1,1.0,0.0,13


In [44]:
matrix = matrix.merge(mu_buy_actions_brand_tmp, on=['user_id'], how='left')
del mu_buy_actions_brand_tmp,mu_buy_actions_brand
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,mu_agg_day_min,mu_agg_day_max,mu_agg_day_mean,mu_agg_day_std,mu_agg_day_sum,mu_agg_brand_min,mu_agg_brand_max,mu_agg_brand_mean,mu_agg_brand_std,mu_agg_brand_sum
0,34176,3906,0,train,451,256,45,109,108,174,...,1,6,1.545455,1.299350,34,1,1,1.0,0.0,22
1,34176,121,0,train,451,256,45,109,108,174,...,1,6,1.545455,1.299350,34,1,1,1.0,0.0,22
2,34176,4356,1,train,451,256,45,109,108,174,...,1,6,1.545455,1.299350,34,1,1,1.0,0.0,22
3,34176,2217,0,train,451,256,45,109,108,174,...,1,6,1.545455,1.299350,34,1,1,1.0,0.0,22
4,230784,4818,0,train,54,31,17,20,19,163,...,1,2,1.166667,0.408248,7,1,1,1.0,0.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,1,2,1.238095,0.436436,26,1,1,1.0,0.0,21
522337,97919,2341,-1,test,55,29,14,17,17,138,...,1,4,2.000000,1.414214,8,1,1,1.0,0.0,4
522338,97919,3971,-1,test,55,29,14,17,17,138,...,1,4,2.000000,1.414214,8,1,1,1.0,0.0,4
522339,32639,3536,-1,test,72,46,24,33,35,172,...,1,1,1.000000,0.000000,8,1,1,1.0,0.0,8


In [45]:
# Merchant aggregation features-day
mu_buy_actions_item = buy_actions.groupby(['user_id','merchant_id'], as_index=False)['item_id'].agg({'mu_agg_item': 'nunique'}).fillna(0)
print(mu_buy_actions_item.head(5))
mu_buy_actions_item_tmp = mu_buy_actions_item.groupby('user_id')['mu_agg_item'].agg([('mu_agg_item_min', 'min'), ('mu_agg_item_max', 'max'),('mu_agg_item_mean', 'mean'),('mu_agg_item_std', 'std'), ('mu_agg_item_sum', 'sum')]).reset_index().fillna(0)
mu_buy_actions_item_tmp.head(5)

  user_id merchant_id  mu_agg_item
0       1        1019            1
1       1        4026            1
2       1         925            1
3      10        1364            3
4      10         938            2


,user_id,mu_agg_item_min,mu_agg_item_max,mu_agg_item_mean,mu_agg_item_std,mu_agg_item_sum
0,1,1,1,1.000000,0.000000,3
1,10,2,3,2.500000,0.707107,5
2,100,1,4,1.500000,1.224745,9
3,1000,1,3,1.400000,0.894427,7
4,10000,1,2,1.076923,0.277350,14


In [46]:
matrix = matrix.merge(mu_buy_actions_item_tmp, on=['user_id'], how='left')
del mu_buy_actions_item_tmp,mu_buy_actions_item
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,mu_agg_brand_min,mu_agg_brand_max,mu_agg_brand_mean,mu_agg_brand_std,mu_agg_brand_sum,mu_agg_item_min,mu_agg_item_max,mu_agg_item_mean,mu_agg_item_std,mu_agg_item_sum
0,34176,3906,0,train,451,256,45,109,108,174,...,1,1,1.0,0.0,22,1,3,1.181818,0.501081,26
1,34176,121,0,train,451,256,45,109,108,174,...,1,1,1.0,0.0,22,1,3,1.181818,0.501081,26
2,34176,4356,1,train,451,256,45,109,108,174,...,1,1,1.0,0.0,22,1,3,1.181818,0.501081,26
3,34176,2217,0,train,451,256,45,109,108,174,...,1,1,1.0,0.0,22,1,3,1.181818,0.501081,26
4,230784,4818,0,train,54,31,17,20,19,163,...,1,1,1.0,0.0,6,1,2,1.166667,0.408248,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,1,1,1.0,0.0,21,1,2,1.190476,0.402374,25
522337,97919,2341,-1,test,55,29,14,17,17,138,...,1,1,1.0,0.0,4,1,3,1.750000,0.957427,7
522338,97919,3971,-1,test,55,29,14,17,17,138,...,1,1,1.0,0.0,4,1,3,1.750000,0.957427,7
522339,32639,3536,-1,test,72,46,24,33,35,172,...,1,1,1.0,0.0,8,1,1,1.000000,0.000000,8


In [72]:
# user_features = pd.read_csv('user_features.csv')
# # user_features['user_id'] = user_features['user_id'].astype(str)
# user_features = user_features.iloc[:,0:11]
# mer_features = pd.read_csv('merchant_features.csv')
# # mer_features['merchant_id'] = mer_features['merchant_id'].astype(str)
# mer_features = mer_features.iloc[:,0:11]
# matrix = matrix.merge(user_features, on = 'user_id',how = "left")
# matrix = matrix.merge(mer_features, on = 'merchant_id',how = "left")
# matrix.to_csv('matrix_new_10LDA.csv')
# matrix

,Unnamed: 0,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,...,mer_LDA_1_y,mer_LDA_2_y,mer_LDA_3_y,mer_LDA_4_y,mer_LDA_5_y,mer_LDA_6_y,mer_LDA_7_y,mer_LDA_8_y,mer_LDA_9_y,mer_LDA_10_y
0,0,34176,3906,0,train,451,256,45,109,108,...,0.000009,0.000009,0.000009,0.020769,0.000009,0.830498,0.000009,0.000639,0.000009,0.000009
1,1,34176,121,0,train,451,256,45,109,108,...,0.005491,0.000245,0.000005,0.002337,0.000005,0.005037,0.023895,0.000005,0.000005,0.515868
2,2,34176,4356,1,train,451,256,45,109,108,...,0.011773,0.176703,0.000022,0.605777,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022
3,3,34176,2217,0,train,451,256,45,109,108,...,0.000003,0.122012,0.000137,0.022823,0.000155,0.215741,0.000003,0.000270,0.000003,0.000003
4,4,230784,4818,0,train,54,31,17,20,19,...,0.000007,0.000007,0.000007,0.238517,0.001242,0.000007,0.000007,0.000952,0.000896,0.009927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,522336,228479,3111,-1,test,2004,1173,71,278,282,...,0.081540,0.021304,0.000012,0.142048,0.000012,0.008272,0.067964,0.000012,0.000012,0.032961
522337,522337,97919,2341,-1,test,55,29,14,17,17,...,0.749279,0.000032,0.000032,0.000032,0.000032,0.000032,0.196710,0.000032,0.001561,0.000032
522338,522338,97919,3971,-1,test,55,29,14,17,17,...,0.000007,0.000007,0.000578,0.144933,0.000007,0.000007,0.000007,0.000007,0.000007,0.003582
522339,522339,32639,3536,-1,test,72,46,24,33,35,...,0.000010,0.000010,0.000010,0.238307,0.000367,0.167526,0.000010,0.019900,0.000010,0.007532


In [73]:
mer_features

,merchant_id,mer_LDA_1,mer_LDA_2,mer_LDA_3,mer_LDA_4,mer_LDA_5,mer_LDA_6,mer_LDA_7,mer_LDA_8,mer_LDA_9,mer_LDA_10
0,1,0.030895,0.000002,0.000002,0.003833,0.000002,0.031226,0.036841,0.000439,0.000002,0.294238
1,10,0.000009,0.000009,0.000009,0.122648,0.000009,0.048455,0.000768,0.000009,0.000009,0.021691
2,100,0.000027,0.035280,0.000027,0.738312,0.000027,0.000027,0.000027,0.000027,0.000027,0.000027
3,1000,0.127581,0.000010,0.000010,0.165325,0.000561,0.005574,0.000010,0.007442,0.000010,0.041086
4,1001,0.000047,0.000047,0.000047,0.040890,0.000047,0.000047,0.000047,0.000047,0.000047,0.173730
...,...,...,...,...,...,...,...,...,...,...,...
4990,995,0.007699,0.594607,0.000026,0.167014,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026
4991,996,0.000063,0.000063,0.000063,0.015179,0.000063,0.599331,0.000063,0.040855,0.000063,0.021357
4992,997,0.000025,0.000025,0.000025,0.066162,0.000025,0.296393,0.000025,0.000025,0.000025,0.000025
4993,998,0.000037,0.148423,0.000037,0.540904,0.000037,0.028190,0.000037,0.000037,0.000037,0.000037


In [74]:
# matrix1 = matrix1.merge(user_features, on = 'user_id',how = "left")
# matrix1 = matrix1.merge(mer_features, on = 'merchant_id',how = "left")
# matrix1.to_csv('matrix_new_10LDA.csv')
# matrix1

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,mer_LDA_1,mer_LDA_2,mer_LDA_3,mer_LDA_4,mer_LDA_5,mer_LDA_6,mer_LDA_7,mer_LDA_8,mer_LDA_9,mer_LDA_10
0,34176,3906,0,train,451,256,45,109,108,174,...,0.000009,0.000009,0.000009,0.020769,0.000009,0.830498,0.000009,0.000639,0.000009,0.000009
1,34176,121,0,train,451,256,45,109,108,174,...,0.005491,0.000245,0.000005,0.002337,0.000005,0.005037,0.023895,0.000005,0.000005,0.515868
2,34176,4356,1,train,451,256,45,109,108,174,...,0.011773,0.176703,0.000022,0.605777,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022
3,34176,2217,0,train,451,256,45,109,108,174,...,0.000003,0.122012,0.000137,0.022823,0.000155,0.215741,0.000003,0.000270,0.000003,0.000003
4,230784,4818,0,train,54,31,17,20,19,163,...,0.000007,0.000007,0.000007,0.238517,0.001242,0.000007,0.000007,0.000952,0.000896,0.009927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,-1,test,2004,1173,71,278,282,184,...,0.081540,0.021304,0.000012,0.142048,0.000012,0.008272,0.067964,0.000012,0.000012,0.032961
522337,97919,2341,-1,test,55,29,14,17,17,138,...,0.749279,0.000032,0.000032,0.000032,0.000032,0.000032,0.196710,0.000032,0.001561,0.000032
522338,97919,3971,-1,test,55,29,14,17,17,138,...,0.000007,0.000007,0.000578,0.144933,0.000007,0.000007,0.000007,0.000007,0.000007,0.003582
522339,32639,3536,-1,test,72,46,24,33,35,172,...,0.000010,0.000010,0.000010,0.238307,0.000367,0.167526,0.000010,0.019900,0.000010,0.007532


# 训练集和测试集划分

In [48]:
# 分割训练数据和测试数据
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')
# matrix.to_csv('matrix_feature.csv')
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']
del temp, matrix
gc.collect()

39

In [66]:
matrix = pd.read_csv('matrix_feature.csv')
matrix1 = matrix.iloc[:,1:141]
print(matrix1.head(5))


   user_id  merchant_id  label origin   u1   u2  u3   u4   u5   u6  ...  \
0    34176         3906      0  train  451  256  45  109  108  174  ...   
1    34176          121      0  train  451  256  45  109  108  174  ...   
2    34176         4356      1  train  451  256  45  109  108  174  ...   
3    34176         2217      0  train  451  256  45  109  108  174  ...   
4   230784         4818      0  train   54   31  17   20   19  163  ...   

   mu_agg_brand_min  mu_agg_brand_max  mu_agg_brand_mean  mu_agg_brand_std  \
0                 1                 1                1.0               0.0   
1                 1                 1                1.0               0.0   
2                 1                 1                1.0               0.0   
3                 1                 1                1.0               0.0   
4                 1                 1                1.0               0.0   

   mu_agg_brand_sum  mu_agg_item_min  mu_agg_item_max  mu_agg_item_mean  \
0    

In [75]:
train_data = matrix1[matrix1['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix1[matrix1['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']
# del temp, matrix
gc.collect()

5943

# XGBoost模型训练

In [78]:
# 导入用到的模型包
import sklearn
from sklearn.model_selection import train_test_split

# import lightgbm as lgb
# 将训练集进行切分，20%用于验证
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.2)

In [56]:
# import xgboost as xgb
# # 使用XGBoost
# model = xgb.XGBClassifier(
#     max_depth=8,
#     n_estimators=2000,
#     min_child_weight=200, 
#     colsample_bytree=0.8, 
#     subsample=0.8, 
#     eta=0.01,    
#     seed=42     
# )
# model.fit(
#     X_train, y_train,
#     eval_metric='auc', 
#     eval_set=[(X_train, y_train), (X_valid, y_valid)],
#     verbose=True,
#     #早停法，如果auc在10epoch没有进步就stop
#     early_stopping_rounds=10 
# )
# model.fit(X_train, y_train)

[0]	validation_0-auc:0.65500	validation_1-auc:0.63421
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.66745	validation_1-auc:0.64895
[2]	validation_0-auc:0.67290	validation_1-auc:0.65529
[3]	validation_0-auc:0.67362	validation_1-auc:0.65649
[4]	validation_0-auc:0.67588	validation_1-auc:0.65715
[5]	validation_0-auc:0.67801	validation_1-auc:0.65873
[6]	validation_0-auc:0.68088	validation_1-auc:0.66106
[7]	validation_0-auc:0.68226	validation_1-auc:0.66144
[8]	validation_0-auc:0.68219	validation_1-auc:0.66102
[9]	validation_0-auc:0.68345	validation_1-auc:0.66201
[10]	validation_0-auc:0.68572	validation_1-auc:0.66299
[11]	validation_0-auc:0.68571	validation_1-auc:0.66353
[12]	validation_0-auc:0.68587	validation_1-auc:0.66354
[13]	validation_0-auc:0.68600	validation_1-auc:0.66404
[14]	validation_0-auc:0.68637	validation_1-auc:0.66472
[15]	validation_0-auc:0.6864

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.01, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.00999999978, max_delta_step=0, max_depth=8,
              min_child_weight=200, missing=nan, monotone_constraints='()',
              n_estimators=2000, n_jobs=0, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)

 # 测试集预测

In [58]:
# prob = model.predict_proba(test_data)
# submission['prob'] = pd.Series(prob[:,1])
# # submission.drop(['origin'], axis=1, inplace=True)
# submission.to_csv('prediction_i_4.csv', index=False)

# lightGBM

In [79]:
import lightgbm as lgb
# 使用LightGBM模型
model = lgb.LGBMClassifier(
    num_leaves=51,
    max_depth=10,
    boosting_type='gbdt',
    objective='binary',
    learning_rate=0.015,
    n_estimators=2000,
    subsample=0.75,
    subsample_freq=2,
    reg_lambda=0.28,
    reg_alpha=0.12,
    colsample_bytree=0.8,
    min_child_samples=300,
    min_split_gain=0.1
)
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_metric='auc', 
    early_stopping_rounds=100
)

[1]	training's auc: 0.656467	training's binary_logloss: 0.228917	valid_1's auc: 0.644964	valid_1's binary_logloss: 0.232918
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.663217	training's binary_logloss: 0.228521	valid_1's auc: 0.651049	valid_1's binary_logloss: 0.232563
[3]	training's auc: 0.668384	training's binary_logloss: 0.228154	valid_1's auc: 0.654932	valid_1's binary_logloss: 0.232244
[4]	training's auc: 0.669571	training's binary_logloss: 0.227802	valid_1's auc: 0.656026	valid_1's binary_logloss: 0.231928
[5]	training's auc: 0.673668	training's binary_logloss: 0.22746	valid_1's auc: 0.659815	valid_1's binary_logloss: 0.231625
[6]	training's auc: 0.677885	training's binary_logloss: 0.227139	valid_1's auc: 0.662622	valid_1's binary_logloss: 0.231334
[7]	training's auc: 0.678829	training's binary_logloss: 0.226804	valid_1's auc: 0.663104	valid_1's binary_logloss: 0.231051
[8]	training's auc: 0.679049	training's binary_logloss: 0.226484	valid_

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.015, max_depth=10,
               min_child_samples=300, min_split_gain=0.1, n_estimators=2000,
               num_leaves=51, objective='binary', reg_alpha=0.12,
               reg_lambda=0.28, subsample=0.75, subsample_freq=2)

In [80]:
prob = model.predict_proba(test_data)
submission.drop(['prob'], axis=1, inplace=True)
submission['prob'] = pd.Series(prob[:,1])
# submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('prediction_ii_6.csv', index=False)